In [66]:
# Import libraries 
import pandas as pd
import uuid

In [68]:
# Load the Excel file
file_path = "Dataset for Exam Essay 2 - Data Governance Exam 2025.xlsx"
# Load all sheets
xls = pd.ExcelFile(file_path)
sheet_names = xls.sheet_names  
print(sheet_names)

['Dataset 1 - Survey', 'Dataset 2 - Twitter data', 'Dataset 3 - Found dataset']


In [70]:
# Read all sheets and fix column names
dfs = {}
for sheet in sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, header=None)  # Don't use header row
    df.columns = df.iloc[1]  # Set row 0 as column names
    df = df[2:]  # Drop row 0
    df.columns = df.columns.str.strip()  # Clean spaces
    dfs[sheet] = df.reset_index(drop=True)

In [72]:
# Replace 'Profile name' with 'Name' in 'Dataset 2 - Twitter data'
if 'Dataset 2 - Twitter data' in dfs:
    dfs['Dataset 2 - Twitter data'] = dfs['Dataset 2 - Twitter data'].rename(columns={'Profile name': 'Name'})
    
# Print column names to verify
for sheet in dfs:
    print(f"Columns in {sheet}: {dfs[sheet].columns.tolist()}")

Columns in Dataset 1 - Survey: ['Name', 'Birth year', 'Occupation', 'Education', 'Monthly income (DKK)', 'Political orientation', 'Religious belief', '"How much do you pay for online news subscriptions per month (in DKK)"?', '"On a scale from 1-5, how much do engage with news sources on social media?"']
Columns in Dataset 2 - Twitter data: ['Comment #', 'Name', 'date', 'comment']
Columns in Dataset 3 - Found dataset: ['Name', 'Date of purchase', 'Geolocation', 'Credit card number', 'Telephone number', 'Subscription', 'Number of Other Subscriptions']


In [74]:
# Trim column names (remove spaces) for consistency
for sheet in dfs:
    dfs[sheet].columns = dfs[sheet].columns.str.strip()

# Merge datasets on the 'Name' column, ensuring missing values are NaN
merged_df = dfs[sheet_names[0]]
for sheet in sheet_names[1:]:
    merged_df = pd.merge(merged_df, dfs[sheet], on="Name", how="outer")

In [76]:
# Remove anyone with birth year above 2012 as these are (potentially) below the age of 13 
merged_df = merged_df[merged_df['Birth year'] < 2012]

# Remove the 'comment' column for anyone with birth year above 2006 as these were below 13 at the time of posting
merged_df.loc[merged_df['Birth year'] > 2006, 'comment'] = None  

# Create bins for each decade from 1900 to 2020 (steps of 10 years) to increase anonymity 
bins = list(range(1900, 2031, 10))  

# Create labels for each decade range, and merge the last two bins (2000-2009 and 2010-2019) into one labeled '2000-2020'
labels = [f"{i}-{i+9}" for i in range(1900, 2021, 10)] 

# Apply pd.cut() to the 'birth year' column to assign each year to the appropriate bin
merged_df['birth_year_bracket'] = pd.cut(merged_df['Birth year'], bins=bins, labels=labels, right=True, include_lowest=True)

# Merge 'birth_year_bracket' for people in the 2000-2009 and 2010-2019 ranges to 2000-2011
merged_df['birth_year_bracket'] = merged_df['birth_year_bracket'].replace(
    {'2000-2009': '2000-2011', '2010-2019': '2000-2011'}
)
# Show the updated columns
print(merged_df[['Name', 'birth_year_bracket']].head())

1                 Name birth_year_bracket
0  Alexander Rodriguez          1990-1999
1         Amelia Brown          1960-1969
2       Andrew Juncker          2000-2011
3       Anthony Nguyen          2000-2011
5            Ava Patel          1980-1989


/var/folders/pf/z1vyy6q91hngl8lsry7fc3lw0000gn/T/ipykernel_69971/2131356986.py:17: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  merged_df['birth_year_bracket'] = merged_df['birth_year_bracket'].replace(


In [83]:
# Conversion rate from DKK to USD 
conversion_rate = 0.14  

# Convert 'income' column from DKK to USD
merged_df['income_usd'] = merged_df['Monthly income (DKK)'] * conversion_rate

# Define income brackets
income_bins = [0, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000, 18000, 20000, 22000, 24000, 26000, 28000, 30000, float('inf')]  
income_labels = ["0-2k", "2k-4k", "4k-6k", "6k-8k", "8k-10k", "10k-12k", "12k-14k", "14k-16k", "16k-18k", "18k-20k", "20k-22k", "22k-24k", "24k-26k", "26k-28k", "28k-30k", "30k+"]  # Labels for each range

# Create the 'income_bracket' column based on the USD values
merged_df['income_bracket_usd'] = pd.cut(merged_df['income_usd'], bins=income_bins, labels=income_labels, right=False)

# Convert the 'How_much_do_you_pay_for_online_news_subscriptions_per_month_(in_DKK)' column from DKK to USD
merged_df['Price_of_subscription_in_USD'] = merged_df['"How much do you pay for online news subscriptions per month (in DKK)"?'] * conversion_rate

# Show the updated columns
print(merged_df[['Name', 'income_bracket_usd', 'Price_of_subscription_in_USD']].head())

1                 Name income_bracket_usd Price_of_subscription_in_USD
0  Alexander Rodriguez              4k-6k                         0.14
1         Amelia Brown              4k-6k                        14.56
2       Andrew Juncker               0-2k                         4.06
3       Anthony Nguyen               0-2k                        32.34
5            Ava Patel              4k-6k                        22.68


In [85]:
# Create a function to generate a random code for each name
def generate_random_code():
    return str(uuid.uuid4())  

# Anonymize the 'Name' column by mapping each unique name to a random code
merged_df['Anonymized_Name'] = merged_df['Name'].apply(lambda x: generate_random_code())

# Show the updated columns
print(merged_df[['Name', 'Anonymized_Name']].head())

1                 Name                       Anonymized_Name
0  Alexander Rodriguez  46b400a5-7199-45a2-89a8-fa98a82a1c77
1         Amelia Brown  6544cdab-5fc2-4f44-ae8c-034d499f7ee0
2       Andrew Juncker  594dfc78-9732-490e-bf25-3e9b521ea441
3       Anthony Nguyen  749f5199-55fd-40a6-9eb0-aad66b5a776c
5            Ava Patel  77bc6f49-e4cd-4a24-859b-9e201cd01a20


In [87]:
# Paraphrase comments 
comment_replacements = {
    "I love this news site. It makes everything easier.": "I really enjoy this news website. It simplifies things a lot.",
    "I use it to entertain my kids, who are 5 and 6 years of age.": "I use it to keep my kids entertained.",
    "I hate this news outlet.": "I'm not a fan of this news outlet.",
    "This is a great sight, I really like it, and it supports my political beliefs as a conservative.": "This site is fantastic; I really appreciate it, and it aligns with my conservative views.",
    "I can surely recommend it for other liberals out there!": "I would definitely recommend it to other liberals!",
    "Great site, five stars!": "Awesome site, I give it five stars!",
    "Great, I love it.": "Awesome, I really enjoy it.",
    "Being a socialist by heart, i really recommend this site.": "As a passionate socialist, I highly recommend this site.",
    "Does the app work for anyone else?": "Does the application work for others?",
    "Can someone recommend other news sites better than this one?": "Can anyone suggest better news sites than this one?",
    "Works fine, but the features don't align with my religious beliefs as a muslim.": "It works well, but the features don’t match my Muslim beliefs.",
    "Can someone tell me if there is a way to filter the news stories?": "Can someone let me know if there’s a way to filter the news articles?"
}

# Apply the replacements to the 'comment' column
merged_df['Comment_paraphrased'] = merged_df['comment'].replace(comment_replacements)

# Print the updated dataframe's first few rows to verify
print(merged_df[['Comment_paraphrased']].head())


1                                Comment_paraphrased
0  As a passionate socialist, I highly recommend ...
1                                                NaN
2                                                NaN
3  I really enjoy this news website. It simplifie...
5              I use it to keep my kids entertained.


In [91]:
# Remove unecessary data 
columns_to_drop = ['Date of purchase', 'Geolocation', 'Credit card number', 'income_usd', 
                   '"How much do you pay for online news subscriptions per month (in DKK)"?', 
                   'Telephone number', 'Number of Other Subscriptions', 'date', 'Monthly income (DKK)', 
                   'Name', 'Birth year', 'comment']
merged_df.drop(columns=columns_to_drop, inplace=True)

In [99]:
# Replace # with Number
merged_df.columns = merged_df.columns.str.replace('#', 'Number')  
# Simplify column name
merged_df.columns = merged_df.columns.str.replace('on_a_scale_from_1-5,_how_much_do_engage_with_news_sources_on_social_media?', 'Extent_of_Engagement')  
# Capitalize column names
merged_df.columns = merged_df.columns.str.capitalize()  
# Remove double quotes from all column names
merged_df.columns = merged_df.columns.str.replace('"', '')
# Replace spaces with underscores in all column names of the merged dataframe
merged_df.columns = merged_df.columns.str.replace(' ', '_')
# Move 'Anonymized Name' to the first column
cols = ['Anonymized_name'] + [col for col in merged_df if col != 'Anonymized_name']
merged_df = merged_df[cols]
# Display the updated column names
print(merged_df.columns)

Index(['Anonymized_name', 'Occupation', 'Education', 'Political_orientation',
       'Religious_belief', 'Extent_of_engagement', 'Comment_number',
       'Subscription', 'Birth_year_bracket', 'Income_bracket_usd',
       'Price_of_subscription_in_usd', 'Comment_paraphrased'],
      dtype='object', name=1)


In [101]:
# Display first five rows of the clean dataframe 
merged_df.head()

1,Anonymized_name,Occupation,Education,Political_orientation,Religious_belief,Extent_of_engagement,Comment_number,Subscription,Birth_year_bracket,Income_bracket_usd,Price_of_subscription_in_usd,Comment_paraphrased
0,46b400a5-7199-45a2-89a8-fa98a82a1c77,Retired,University,Liberal,Atheist,1,Comment 13,Wired,1990-1999,4k-6k,0.14,"As a passionate socialist, I highly recommend ..."
1,6544cdab-5fc2-4f44-ae8c-034d499f7ee0,Retired,Vocational training,Conservative,Lutheran,3,NaN,Financial Times,1960-1969,4k-6k,14.56,NaN
2,594dfc78-9732-490e-bf25-3e9b521ea441,University Student,High school,NaN,Buddhist,2,NaN,WashingtonPost,2000-2011,0-2k,4.06,NaN
3,749f5199-55fd-40a6-9eb0-aad66b5a776c,University Student,High school,NaN,Atheist,4,Comment 1,NYT,2000-2011,0-2k,32.34,I really enjoy this news website. It simplifie...
5,77bc6f49-e4cd-4a24-859b-9e201cd01a20,Military officer,Millitary school,Socialist,Catholic,3,Comment 3,Politiken,1980-1989,4k-6k,22.68,I use it to keep my kids entertained.


In [ ]:
# Save the cleaned DataFrame to an Excel file
merged_df.to_excel("appendix1_with_names.xlsx", index=False)